# Create data files 
Takes the data from the `station_means.txt` and combines it with the `"Stations info".xlsx`to create to set of files. 
### A. Mapping
- MAPPING : File | Area | BFI | Mean Elevation | Datum[East, North] 

### B. Soil information
- SOIL IN : File | Name | BFI  | Landuse[Arti, agri, forest, wetl, water]\(%\) | Soil[1, 2, 3, 4, 5]\(%\)

In [87]:
import pandas as pd 
import os 
import sys 
import numpy as np 
from typing import List, Dict

In [88]:
columns_mapping = [
    "File",
    "Name",
    "Area",
    "BFI",
    "Elevation mean",
    "Datum",
    "East",
    "North"
]


# S : Soil infiltration capacity
columns_soilin = [
    "File",
    "Name",
    "BFI",
    "Artificial",
    "Agriculture",
    "Wetland",
    "Waterbody",
    "S1",          # SL_IFT1: Well suited (%)
    "S2",          # SL_IFT2: Medium suited (%)
    "S3",          # SL_IFT3: Little suited (%)
    "S4",          # SL_IFT4: Unsuitable (%)
    "S5"           # SL_IFT5 (Not classified%)
]

In [89]:
def mapping_to_file(finalpath: str, df: pd.DataFrame) -> None:
    filetype = ".txt"
    datums = ["UTM32", "UTM33", "UTM34", "UTM35", "UTM36"]
    for datum in datums:
        utm = df[df["Datum"] == datum]
        utm.to_csv(finalpath + datum + filetype, index=False)

In [90]:
def soilin_to_file(finalpath: str, df: pd.DataFrame) -> None:
    filetype = ".csv"
    name = "SoilInfitrationCapacity"
    df.to_csv(finalpath + name + filetype, index=False)

In [91]:
def extract_columns(col : List[str], df) -> Dict[str, pd.Series]:
    d = {}
    for key in col:
        if key in df.columns:
            d[key] = df[key]
            return d                        


In [92]:
df_sm : pd.DataFrame # Station means dataframe
path_sm : str = "../GEO3000/code/data/lh_0.925_final/station_means.txt"
df_sm = pd.read_csv(path_sm, delim_whitespace=True)
    
    
df_si : pd.DataFrame # station info
path_si : str = "./data/Landuse and soil data.xlsx"
df_si = pd.read_excel(path_si, header=1, dtype={"station": str})

In [93]:
# display(df_sm)
df_sm.drop(["Period", "Completness"], axis=1, inplace=True, errors="ignore")
df_sm["File"] = df_sm["File"].apply(lambda x: x[:-2])

In [94]:
df_1 = df_sm.copy()
df_1.set_index("File", inplace=True)

In [95]:
# df_si.drop(["elev_mean"], axis=1, inplace=True, errors="ignore")
display(df_sm) 

,File,Name,Area,BFI,Datum,East,North
0,2.634,Lena,183.6,0.472136,UTM32,271220,6733473
1,12.209,Urula,554.1,0.523131,UTM32,219200,6724518
2,105.1,Osenelv,137.6,0.515502,UTM32,129534,6983503
3,88.4,Lovatn,234.9,0.644546,UTM32,74031,6885087
4,2.28,Aulestad,869.8,0.492369,UTM32,246027,6795979
...,...,...,...,...,...,...,...
95,22.22,Sogne,203.6,0.436342,UTM32,78051,6461281
96,83.2,Viksvatn,508.1,0.633612,UTM32,13361,6833950
97,2.145,Losna,11205.4,0.681366,UTM32,247417,6809063
98,247.3,Karpelva,128.9,0.526866,UTM36,1091325,7802944


In [96]:
stations_with_info = df_si["station"] # Creates a series of all the stations of interest. 
stations_total = df_sm["File"]
c = 0
for ele in stations_with_info:
    if ele in stations_total.values:
        c += 1
c

52

In [97]:
na = df_si["name"]
nb = df_sm["Name"]
c = 0
for ele in nb:
    if ele.lower() in na.values:
        c+=1      
c

52

In [98]:
df_1["Elevation mean"] = -9999

In [99]:
df_2 = df_si.copy()
df_2.set_index("station", inplace=True)
c = 0
for s in stations_with_info:
    if s in stations_total.values:
        df_1["Elevation mean"].loc[s] = df_2["elev_mean"].loc[s]

/home/sigurd/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [100]:
df_1.reset_index(inplace=True)
df_1 = df_1[columns_mapping]
df_1

,File,Name,Area,BFI,Elevation mean,Datum,East,North
0,2.634,Lena,183.6,0.472136,453,UTM32,271220,6733473
1,12.209,Urula,554.1,0.523131,806,UTM32,219200,6724518
2,105.1,Osenelv,137.6,0.515502,252,UTM32,129534,6983503
3,88.4,Lovatn,234.9,0.644546,-9999,UTM32,74031,6885087
4,2.28,Aulestad,869.8,0.492369,-9999,UTM32,246027,6795979
...,...,...,...,...,...,...,...,...
95,22.22,Sogne,203.6,0.436342,196,UTM32,78051,6461281
96,83.2,Viksvatn,508.1,0.633612,813,UTM32,13361,6833950
97,2.145,Losna,11205.4,0.681366,-9999,UTM32,247417,6809063
98,247.3,Karpelva,128.9,0.526866,-9999,UTM36,1091325,7802944


In [117]:
finalpath_mapping: str = "../GEO3000/code/data/map/Q"
a = df_1[df_1["Elevation mean"] == -9999]
b = df_1[df_1["Elevation mean"] != -9999]
# mapping_to_file(finalpath_mapping, a)
mapping_to_file(finalpath_mapping, b)
display(df_1)

,File,Name,Area,BFI,Elevation mean,Datum,East,North
0,2.634,Lena,183.6,0.472136,453,UTM32,271220,6733473
1,12.209,Urula,554.1,0.523131,806,UTM32,219200,6724518
2,105.1,Osenelv,137.6,0.515502,252,UTM32,129534,6983503
3,88.4,Lovatn,234.9,0.644546,-9999,UTM32,74031,6885087
4,2.28,Aulestad,869.8,0.492369,-9999,UTM32,246027,6795979
...,...,...,...,...,...,...,...,...
95,22.22,Sogne,203.6,0.436342,196,UTM32,78051,6461281
96,83.2,Viksvatn,508.1,0.633612,813,UTM32,13361,6833950
97,2.145,Losna,11205.4,0.681366,-9999,UTM32,247417,6809063
98,247.3,Karpelva,128.9,0.526866,-9999,UTM36,1091325,7802944


In [102]:
print(columns_mapping)
print(columns_soilin)

['File', 'Name', 'Area', 'BFI', 'Elevation mean', 'Datum', 'East', 'North']
['File', 'Name', 'BFI', 'Artificial', 'Agriculture', 'Wetland', 'Waterbody', 'S1', 'S2', 'S3', 'S4', 'S5']


In [114]:
df_test = df_1.copy()
a = df_test[df_test["Elevation mean"] == -9999]

In [115]:
a.reset_index()
a

,File,Name,Area,BFI,Elevation mean,Datum,East,North
3,88.4,Lovatn,234.9,0.644546,-9999,UTM32,74031,6885087
4,2.28,Aulestad,869.8,0.492369,-9999,UTM32,246027,6795979
10,78.8,Boyumselv,40.5,0.620123,-9999,UTM32,60541,6840653
11,124.2,Hoggaasbru,494.5,0.516690,-9999,UTM32,318730,7045669
12,121.20,Aamot,282.7,0.554068,-9999,UTM32,238900,7011786
14,212.10,Masi,5620.8,0.594211,-9999,UTM34,837785,7725086
15,122.9,Gaulfoss,3085.9,0.513347,-9999,UTM32,259399,7006617
16,26.29,Refsvatn,53.0,0.453220,-9999,UTM32,-5758,6500337
17,2.604,Elverum,15449.9,0.668329,-9999,UTM32,313304,6753674
18,103.40,Horgheim,1099.4,0.593914,-9999,UTM32,128391,6947792
